In [12]:
import re
import itertools
from pathlib import Path
import scipy
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [13]:
data_path = Path('../data/2975_LickingLama_20250207_125807.mat')
data = scipy.io.loadmat(data_path)
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'SessionData'])

`data['Vc']` contains wide-field imaging time-series, `data['zeromeanVC']` probably is some scaled version of the same data.

In [15]:
import h5py

vc_path = Path('../data/Vc_2975.mat')
print(f"Loading variables from {vc_path}")

with h5py.File(vc_path, 'r') as f:
    # Load the specific variables
    Vc = f['Vc'][:]
    trialOn = f['trialOn'][:]
    U = f['U'][:]
    ledInfo = f['ledInfo'][:]  # Load as numpy array
    frameInfo = f['frameInfo'][:]  # Load as numpy array
    
    print(f"Vc shape: {Vc.shape}")
    print(f"trialOn shape: {trialOn.shape}")
    print(f"U shape: {U.shape}")
    print(f"ledInfo shape: {ledInfo.shape}")
    print(f"frameInfo shape: {frameInfo.shape}")  # Should be (39341, 2)

Loading variables from ..\data\Vc_2975.mat
Vc shape: (41623, 500)
trialOn shape: (3, 1000000)
U shape: (500, 640, 540)
ledInfo shape: (3, 41623)
frameInfo shape: (2, 41623)


In [16]:
print('Vc', Vc.shape)

Vc (41623, 500)


In [5]:
plt.title(f"dimensions: {data['Vm'].shape[0]} (components), {data['Vm'].shape[1]} (frames)")
plt.plot(data['zeromeanVc'][0, :1000], label="zeromeanVC")
plt.plot(data['Vm'][0, :1000], label="Vm")
plt.legend()

KeyError: 'Vm'

Each components is a weighted combination of pixels, weights are stored in `data["U"]`

In [ ]:
data["U"].shape

In [ ]:
plt.imshow(data["U"][:, :, 0])

`data['R']` contains the design matrix.

In [ ]:
plt.imshow(data['R'][:500].T, aspect="auto")
plt.title(f'dimensions: {data["R"].shape[0]} (frames), {data["R"].shape[1]} (time-lagged features)')

Different regressors are stored in `data["regLabels"]` and `data["regIdx"]` maps the regressors to the design matrix

In [ ]:
data["regLabels"].shape, data["regIdx"].shape, np.unique(data["regIdx"]).size

data `["lambdas"]` (probably) contains the best $\lambda$ value for every component.

In [ ]:
data["betas"].shape, data["lambdas"].shape

In [ ]:
plt.hist(data["lambdas"][0])

In [ ]:
len(design_df_columns), len(reg_idx)

What is this?

In [ ]:
data["cMovie"].shape, data["cMap"].shape

The model weights are contained in `data["betas"]` there are 10 (probably CV folds) arrays shaped 2435 x 200

In [ ]:
data["betas"].shape, data["betas"][0][0].shape

Average across cross-validation folds

In [ ]:
beta = np.stack(data["betas"].flatten()).mean(axis=0)
beta.shape

Plot weights for 1 regressor

In [ ]:
feature = 1
mask = data["regIdx"] == feature
plt.plot(beta[mask.flatten(), 0:10]);